**SENTIMENT ANALYSIS**

In [ ]:
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jp797498e/twitter-entity-sentiment-analysis")

print("Path to dataset files:", path)

# Assuming the dataset is a CSV file, load it into a Pandas DataFrame
df = pd.read_csv(path + '/twitter_training.csv') # Load data into a dataframe, assuming your csv is in the default dataset path


100%|██████████| 1.99M/1.99M [00:01<00:00, 1.81MB/s]

Extracting files...


Path to dataset files: /root/.cache/kagglehub/datasets/jp797498e/twitter-entity-sentiment-analysis/versions/2


In [ ]:
df.head()

,2401,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,2401,Borderlands,Positive,I am coming to the borders and I will kill you...
1,2401,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,2401,Borderlands,Positive,im coming on borderlands and i will murder you...
3,2401,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,2401,Borderlands,Positive,im getting into borderlands and i can murder y...


Deleting unwanted columns

In [ ]:
df.drop(columns = ['2401'], inplace = True)

In [ ]:
df.head()

,Borderlands,Positive,"im getting on borderlands and i will murder you all ,"
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...


Renaming columns

In [ ]:
df.columns

Index(['Borderlands', 'Positive',
       'im getting on borderlands and i will murder you all ,'],
      dtype='object')

In [ ]:
df.rename(columns={'Borderlands' :'Game_name',
            'Positive' : 'Sentiment',
           'im getting on borderlands and i will murder you all ,' : 'comment'}, inplace = True)

In [ ]:
df.head()

,Game_name,Sentiment,comment
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...


Handling missing values

In [ ]:
df.isna().sum()

,0
Game_name,0
Sentiment,0
comment,686


In [ ]:
df.shape

(74681, 3)

In [ ]:
df = df.dropna()

In [ ]:
df.isna().sum()

,0
Game_name,0
Sentiment,0
comment,0


Creating a new dataset

In [ ]:
df.Sentiment.value_counts()

,count
Sentiment,
Negative,22358
Positive,20654
Neutral,18108
Irrelevant,12875


In [ ]:
df_positive = df[df.Sentiment == 'Positive'].iloc[:2000]
df_negative = df[df.Sentiment == 'Negative'].iloc[:2000]
df_neutral = df[df.Sentiment == 'Neutral'].iloc[:2000]
df_irr = df[df.Sentiment == 'Irrelevant'].iloc[:2000]

In [ ]:
df2 = pd.concat([df_positive, df_negative, df_neutral, df_irr],axis=0)

In [ ]:
df2.shape

(8000, 3)

In [ ]:
df2.Sentiment.value_counts()

,count
Sentiment,
Positive,2000
Negative,2000
Neutral,2000
Irrelevant,2000


In [ ]:
df2.head()

,Game_name,Sentiment,comment
0,Borderlands,Positive,I am coming to the borders and I will kill you...
1,Borderlands,Positive,im getting on borderlands and i will kill you ...
2,Borderlands,Positive,im coming on borderlands and i will murder you...
3,Borderlands,Positive,im getting on borderlands 2 and i will murder ...
4,Borderlands,Positive,im getting into borderlands and i can murder y...


Categorical to numerical data conversion

In [ ]:
df2.Game_name.unique()

array(['Borderlands', 'CallOfDutyBlackopsColdWar', 'Amazon', 'Overwatch',
       'Xbox(Xseries)'], dtype=object)

In [ ]:
game_df = pd.get_dummies(df2['Game_name']).astype('int')

In [ ]:
game_df

,Amazon,Borderlands,CallOfDutyBlackopsColdWar,Overwatch,Xbox(Xseries)
0,0,1,0,0,0
1,0,1,0,0,0
2,0,1,0,0,0
3,0,1,0,0,0
4,0,1,0,0,0
...,...,...,...,...,...
10628,0,0,0,0,1
10629,0,0,0,0,1
10630,0,0,0,0,1
10631,0,0,0,0,1


In [ ]:
df2.drop(columns = ['Game_name'], inplace = True)

In [ ]:
df2.head()

,Sentiment,comment
0,Positive,I am coming to the borders and I will kill you...
1,Positive,im getting on borderlands and i will kill you ...
2,Positive,im coming on borderlands and i will murder you...
3,Positive,im getting on borderlands 2 and i will murder ...
4,Positive,im getting into borderlands and i can murder y...


In [ ]:
df2.shape , game_df.shape

((8000, 2), (8000, 5))

Text preprocessing

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
def lemmatization(text):
    doc = nlp(text)
    lemmalist = [word.lemma_ for word in doc]
    return " ".join([token.lemma_ for token in doc])

In [ ]:
df2['lemma'] = df2['comment'].apply(lemmatization)

In [ ]:
df2.head()

,Sentiment,comment,lemma
0,Positive,I am coming to the borders and I will kill you...,"I be come to the border and I will kill you all ,"
1,Positive,im getting on borderlands and i will kill you ...,"I m get on borderland and I will kill you all ,"
2,Positive,im coming on borderlands and i will murder you...,I m come on borderland and I will murder you a...
3,Positive,im getting on borderlands 2 and i will murder ...,I m get on borderland 2 and I will murder you ...
4,Positive,im getting into borderlands and i can murder y...,I m get into borderland and I can murder you a...


In [ ]:
def remove_stopwords(text):
    doc = nlp(text)
    no_stop_words = [token.text for token in doc if not token.is_stop]
    return " ".join(no_stop_words)

In [ ]:
df2['final'] = df2['lemma'].apply(remove_stopwords)

In [ ]:
df2.head()

,Sentiment,comment,lemma,final
0,Positive,I am coming to the borders and I will kill you...,"I be come to the border and I will kill you all ,","come border kill ,"
1,Positive,im getting on borderlands and i will kill you ...,"I m get on borderland and I will kill you all ,","m borderland kill ,"
2,Positive,im coming on borderlands and i will murder you...,I m come on borderland and I will murder you a...,"m come borderland murder ,"
3,Positive,im getting on borderlands 2 and i will murder ...,I m get on borderland 2 and I will murder you ...,"m borderland 2 murder ,"
4,Positive,im getting into borderlands and i can murder y...,I m get into borderland and I can murder you a...,"m borderland murder ,"


In [ ]:
df2.drop(columns = ['comment', 'lemma'], inplace = True)

In [ ]:
df2.head()

,Sentiment,final
0,Positive,"come border kill ,"
1,Positive,"m borderland kill ,"
2,Positive,"m come borderland murder ,"
3,Positive,"m borderland 2 murder ,"
4,Positive,"m borderland murder ,"


In [ ]:
sentiment_df = pd.concat([df2, game_df], axis = 1)

In [ ]:
sentiment_df.head()

,Sentiment,final,Amazon,Borderlands,CallOfDutyBlackopsColdWar,Overwatch,Xbox(Xseries)
0,Positive,"come border kill ,",0,1,0,0,0
1,Positive,"m borderland kill ,",0,1,0,0,0
2,Positive,"m come borderland murder ,",0,1,0,0,0
3,Positive,"m borderland 2 murder ,",0,1,0,0,0
4,Positive,"m borderland murder ,",0,1,0,0,0


In [ ]:
df2.shape

(8000, 2)

In [ ]:
df2.isna().sum()

,0
Sentiment,0
final,0


Dividing the dataset

In [ ]:
x = sentiment_df.drop(columns = ['Sentiment'])
y = sentiment_df['Sentiment']

In [ ]:
x.head()

,final,Amazon,Borderlands,CallOfDutyBlackopsColdWar,Overwatch,Xbox(Xseries)
0,"come border kill ,",0,1,0,0,0
1,"m borderland kill ,",0,1,0,0,0
2,"m come borderland murder ,",0,1,0,0,0
3,"m borderland 2 murder ,",0,1,0,0,0
4,"m borderland murder ,",0,1,0,0,0


TFIDF Vectorizer

In [ ]:
tfidf = TfidfVectorizer()

In [ ]:
tfidf.fit(x['final'])

TfidfVectorizer()

In [ ]:
tfidf_df = tfidf.transform(x['final'])

In [ ]:
tfidf_matrix = tfidf.transform(x['final']).toarray()

In [ ]:
tfidf_matrix

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
vectorizer_df = pd.DataFrame(tfidf_matrix, columns=tfidf.get_feature_names_out())

In [ ]:
vectorizer_df.head()

,00,000,01,02,03,03573057,0359873057,04,05,06,...,zxxxvids,zyfapoihpy,ееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееее,есть,июля,сетью,третьарце,اللعبه,حبيت,خلاص
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x.drop(columns=['final'], inplace=True)

In [ ]:
x.shape, vectorizer_df.shape

((8000, 5), (8000, 6720))

In [ ]:
x = pd.concat([x, vectorizer_df], axis = 1)

In [ ]:
x.head()

,Amazon,Borderlands,CallOfDutyBlackopsColdWar,Overwatch,Xbox(Xseries),00,000,01,02,03,...,zxxxvids,zyfapoihpy,ееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееееее,есть,июля,сетью,третьарце,اللعبه,حبيت,خلاص
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
x.isna().sum()

,0
Amazon,952
Borderlands,952
CallOfDutyBlackopsColdWar,952
Overwatch,952
Xbox(Xseries),952
...,...
сетью,952
третьарце,952
اللعبه,952
حبيت,952


In [ ]:
x = x.fillna(0)

In [ ]:
x.isna().sum()

,0
Amazon,0
Borderlands,0
CallOfDutyBlackopsColdWar,0
Overwatch,0
Xbox(Xseries),0
...,...
сетью,0
третьарце,0
اللعبه,0
حبيت,0


Train - Test split

In [ ]:
x.shape , y.shape

((8952, 6725), (8000,))

In [ ]:
x = x.iloc[:8000]

In [ ]:
x.shape , y.shape

((8000, 6725), (8000,))

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [ ]:
x_train.shape , x_test.shape , y_train.shape , y_test.shape

((6400, 6725), (1600, 6725), (6400,), (1600,))

Building the model

In [ ]:
model = RandomForestClassifier(n_jobs = 1,
                               random_state = 42,
                               n_estimators = 200)
model.fit(x_train, y_train)

RandomForestClassifier(n_estimators=200, n_jobs=1, random_state=42)

In [ ]:
pred = model.predict(x_test)

In [ ]:
y_test[7:11]

,Sentiment
2674,Positive
210,Neutral
7657,Negative
3704,Positive


In [ ]:
pred[7:11]

array(['Positive', 'Neutral', 'Negative', 'Positive'], dtype=object)

In [ ]:
accuracy_score(y_test, pred) *100

71.0

In [ ]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

  Irrelevant       0.73      0.73      0.73       399
    Negative       0.73      0.59      0.65       379
     Neutral       0.70      0.74      0.72       397
    Positive       0.69      0.78      0.73       425

    accuracy                           0.71      1600
   macro avg       0.71      0.71      0.71      1600
weighted avg       0.71      0.71      0.71      1600

